# Hardware
### DGX Station
V100 SMX2 16GB

Intel Xeon E5-2968 v4 @ 2.20GHz

# Software
CUDA 11.0

cuSignal 0.19

SciPy 1.6.3

Numba 0.53.1

# Profiling
[Nsight Systems](https://docs.nvidia.com/nsight-systems/UserGuide/index.html)

## CPU Baseline
Baseline Scipy's Signal Lombscargle function

In [1]:
!nsys profile -s none -o scipy_v1 -f true --stats=true python3 icassp_scipy_v1.py float32 5

Processing events...
Capturing symbol files...
Saving temporary "/tmp/nsys-report-1ebf-808d-66fd-2f5f.qdstrm" file to disk...
Creating final output files...

Processing [==============================================================100%]
Saved report file to "/tmp/nsys-report-1ebf-808d-66fd-2f5f.qdrep"
Exporting 614774 events: [================================================100%]

Exported successfully to
/tmp/nsys-report-1ebf-808d-66fd-2f5f.sqlite

Generating CUDA API Statistics...
CUDA API Statistics (nanoseconds)




CUDA trace data was not collected.


Generating Operating System Runtime API Statistics...
Operating System Runtime API Statistics (nanoseconds)

Time(%)      Total Time       Calls         Average         Minimum         Maximum  Name                                                                            
-------  --------------  ----------  --------------  --------------  --------------  ----------------------------------------------------------------------------

### Single Precision
|                       | Registers | JIT (ms) | Kernel (ms) | Speed Up |
|:----------------------|:----------|:---------|:------------|:---------|
| Scipy                 | -         | 51,075.2 | 51,128.2    | 1.0      |
| CuPy (Baseline)       |           |          |             |          |
| CuPy (User Cache)     |           |          |             |          |
| CuPy (Data Type)      |           |          |             |          |
| CuPy (Fast Math)      |           |          |             |          |
| CuPy (Fatbin)         |           |          |             |          |
| CuPy (Launch Bounds)  |           |          |             |          |
| CuPy (Trig Functions) |           |          |             |          |

### Double Precision
|                       | Registers | JIT (ms) | Kernel (ms) | Speed Up |
|:----------------------|:----------|:---------|:------------|:---------|
| Scipy                 | -         | 50.799.1 | 50,806.4    | 1.0      |
| CuPy (Baseline)       |           |          |             |          |
| CuPy (User Cache)     |           |          |             |          |
| CuPy (Data Type)      |           |          |             |          |
| CuPy (Fast Math)      |           |          |             |          |
| CuPy (Fatbin)         |           |          |             |          |
| CuPy (Launch Bounds)  |           |          |             |          |
| CuPy (Trig Functions) |           |          |             |          |

## GPU Scenario #1
1. CuPy implementation of Lombscargle function

In [3]:
!nsys profile -s none -o cupy_v1 -f true --stats=true python3 icassp_cupy_v1.py float32 2000 0

Processing events...
Capturing symbol files...
Saving temporary "/tmp/nsys-report-54d8-f4a0-2a49-af96.qdstrm" file to disk...
Creating final output files...

Processing [==============================================================100%]
Saved report file to "/tmp/nsys-report-54d8-f4a0-2a49-af96.qdrep"
Exporting 660402 events: [================================================100%]

Exported successfully to
/tmp/nsys-report-54d8-f4a0-2a49-af96.sqlite

Generating CUDA API Statistics...
CUDA API Statistics (nanoseconds)

Time(%)      Total Time       Calls         Average         Minimum         Maximum  Name                                                                            
-------  --------------  ----------  --------------  --------------  --------------  --------------------------------------------------------------------------------
   97.7     20032673990        2001      10011331.3         9566349        10747533  cudaDeviceSynchronize                                      

In [4]:
#!nsys stats --report gpukernsum --report nvtxppsum --report gputrace cupy_v1.qdrep # Use to get register usage
!nsys stats --report gpukernsum --report nvtxppsum cupy_v1.qdrep

Using cupy_v1.sqlite file for stats and reports.
Exporting [/opt/nvidia/nsight-systems/2020.3.2/target-linux-x64/reports/gpukernsum cupy_v1.sqlite] to console... 

 Time(%)  Total Time (ns)  Instances    Average      Minimum    Maximum          Name       
 -------  ---------------  ---------  ------------  ---------  ----------  -----------------
   100.0   20,030,143,874      2,001  10,010,066.9  9,565,029  10,746,358  _cupy_lombscargle

Exporting [/opt/nvidia/nsight-systems/2020.3.2/target-linux-x64/reports/nvtxppsum cupy_v1.sqlite] to console... 

 Time(%)  Total Time (ns)  Instances     Average       Minimum      Maximum            Range        
 -------  ---------------  ---------  -------------  -----------  -----------  ---------------------
    99.3   20,220,069,274      2,000   10,110,034.6    9,662,342   10,842,798  cupy_lombscargle_loop
     0.7      135,131,738          1  135,131,738.0  135,131,738  135,131,738  cupy_lombscargle     



### Single Precision
|                       | Registers | JIT (ms) | Kernel (ms) | Speed Up |
|:----------------------|:----------|:---------|:------------|:---------|
| Scipy                 | -         | 51,075.2 | 51,128.2    | 1.0      |
| CuPy (Baseline)       | 30        | 135.1    | 10.1        | 5062.2   |
| CuPy (User Cache)     |           |          |             |          |
| CuPy (Data Type)      |           |          |             |          |
| CuPy (Fast Math)      |           |          |             |          |
| CuPy (Fatbin)         |           |          |             |          |
| CuPy (Launch Bounds)  |           |          |             |          |
| CuPy (Trig Functions) |           |          |             |          |

### Double Precision
|                       | Registers | JIT (ms) | Kernel (ms) | Speed Up |
|:----------------------|:----------|:---------|:------------|:---------|
| Scipy                 | -         | 50.799.1 | 50,806.4    | 1.0      |
| CuPy (Baseline)       | 48        | 145.0    | 21.2        | 2396.5   |
| CuPy (User Cache)     |           |          |             |          |
| CuPy (Data Type)      |           |          |             |          |
| CuPy (Fast Math)      |           |          |             |          |
| CuPy (Fatbin)         |           |          |             |          |
| CuPy (Launch Bounds)  |           |          |             |          |
| CuPy (Trig Functions) |           |          |             |          |

## GPU Scenario #2
1. CuPy implementation of Lombscargle function
2. Added user cache

In [9]:
!nsys profile -s none -o cupy_v2 -f true python3 icassp_cupy_v2.py float32 2000 0

Registers 30
Processing events...
Capturing symbol files...
Saving temporary "/tmp/nsys-report-cd79-4fad-c16e-b97d.qdstrm" file to disk...
Creating final output files...

Processing [==============================================================100%]
Saved report file to "/tmp/nsys-report-cd79-4fad-c16e-b97d.qdrep"
Report file moved to "/home/odysseus/workStuff/cusignal-icassp-tutorial/notebooks/raw_cupy_cuda/cupy_v2.qdrep"


In [10]:
!nsys stats --report gpukernsum --report nvtxppsum cupy_v2.qdrep

Generate SQLite file cupy_v2.sqlite from cupy_v2.qdrep
Exporting 627338 events: [================================================100%]
Using cupy_v2.sqlite file for stats and reports.
Exporting [/opt/nvidia/nsight-systems/2020.3.2/target-linux-x64/reports/gpukernsum cupy_v2.sqlite] to console... 

 Time(%)  Total Time (ns)  Instances    Average      Minimum    Maximum          Name       
 -------  ---------------  ---------  ------------  ---------  ----------  -----------------
   100.0   20,058,169,271      2,001  10,024,072.6  9,563,435  11,300,375  _cupy_lombscargle

Exporting [/opt/nvidia/nsight-systems/2020.3.2/target-linux-x64/reports/nvtxppsum cupy_v2.sqlite] to console... 

 Time(%)  Total Time (ns)  Instances     Average       Minimum      Maximum            Range        
 -------  ---------------  ---------  -------------  -----------  -----------  ---------------------
    99.3   20,175,315,289      2,000   10,087,657.6    9,620,977   11,365,222  cupy_lombscargle_loop
    

### Single Precision
|                       | Registers | JIT (ms) | Kernel (ms) | Speed Up |
|:----------------------|:----------|:---------|:------------|:---------|
| Scipy                 | -         | 51,075.2 | 51,128.2    | 1.0      |
| CuPy (Baseline)       | 30        | 135.1    | 10.1        | 5062.2   |
| CuPy (User Cache)     | 30        | 137.2    | 10.1        | 5062.2   |
| CuPy (Data Type)      |           |          |             |          |
| CuPy (Fast Math)      |           |          |             |          |
| CuPy (Fatbin)         |           |          |             |          |
| CuPy (Launch Bounds)  |           |          |             |          |
| CuPy (Trig Functions) |           |          |             |          |

### Double Precision
|                       | Registers | JIT (ms) | Kernel (ms) | Speed Up |
|:----------------------|:----------|:---------|:------------|:---------|
| Scipy                 | -         | 50.799.1 | 50,806.4    | 1.0      |
| CuPy (Baseline)       | 48        | 145.0    | 21.2        | 2396.5   |
| CuPy (User Cache)     | 48        | 145.7    | 21.2        | 2396.5   |
| CuPy (Data Type)      |           |          |             |          |
| CuPy (Fast Math)      |           |          |             |          |
| CuPy (Fatbin)         |           |          |             |          |
| CuPy (Launch Bounds)  |           |          |             |          |
| CuPy (Trig Functions) |           |          |             |          |

## GPU Scenario #3
1. CuPy implementation of Lombscargle function
2. Added user cache
3. Specific functions per data type

In [13]:
!nsys profile -s none -o cupy_v3 -f true python3 icassp_cupy_v3.py float32 2000 0

Registers 26
Processing events...
Capturing symbol files...
Saving temporary "/tmp/nsys-report-94d4-cbfd-f6ad-ff21.qdstrm" file to disk...
Creating final output files...

Processing [==============================================================100%]
Saved report file to "/tmp/nsys-report-94d4-cbfd-f6ad-ff21.qdrep"
Report file moved to "/home/odysseus/workStuff/cusignal-icassp-tutorial/notebooks/raw_cupy_cuda/cupy_v3.qdrep"


In [14]:
!nsys stats --report gpukernsum --report nvtxppsum cupy_v3.qdrep

Generate SQLite file cupy_v3.sqlite from cupy_v3.qdrep
Exporting 655330 events: [================================================100%]
Using cupy_v3.sqlite file for stats and reports.
Exporting [/opt/nvidia/nsight-systems/2020.3.2/target-linux-x64/reports/gpukernsum cupy_v3.sqlite] to console... 

 Time(%)  Total Time (ns)  Instances    Average      Minimum    Maximum              Name           
 -------  ---------------  ---------  ------------  ---------  ----------  -------------------------
   100.0   20,418,466,090      2,001  10,204,131.0  9,710,413  10,914,239  _cupy_lombscargle_float32

Exporting [/opt/nvidia/nsight-systems/2020.3.2/target-linux-x64/reports/nvtxppsum cupy_v3.sqlite] to console... 

 Time(%)  Total Time (ns)  Instances     Average       Minimum      Maximum            Range        
 -------  ---------------  ---------  -------------  -----------  -----------  ---------------------
    99.3   20,600,928,564      2,000   10,300,464.3    9,805,052   11,009,453  cu

### Single Precision
|                       | Registers | JIT (ms) | Kernel (ms) | Speed Up |
|:----------------------|:----------|:---------|:------------|:---------|
| Scipy                 | -         | 51,075.2 | 51,128.2    | 1.0      |
| CuPy (Baseline)       | 30        | 135.1    | 10.1        | 5062.2   |
| CuPy (User Cache)     | 30        | 137.2    | 10.1        | 5062.2   |
| CuPy (Data Type)      | 26        | 135.4    | 10.3        | 4963.9   |
| CuPy (Fast Math)      |           |          |             |          |
| CuPy (Fatbin)         |           |          |             |          |
| CuPy (Launch Bounds)  |           |          |             |          |
| CuPy (Trig Functions) |           |          |             |          |

### Double Precision
|                       | Registers | JIT (ms) | Kernel (ms) | Speed Up |
|:----------------------|:----------|:---------|:------------|:---------|
| Scipy                 | -         | 50.799.1 | 50,806.4    | 1.0      |
| CuPy (Baseline)       | 48        | 145.0    | 21.2        | 2396.5   |
| CuPy (User Cache)     | 48        | 145.7    | 21.2        | 2396.5   |
| CuPy (Data Type)      | 48        | 150.4    | 21.2        | 2396.5   |
| CuPy (Fast Math)      |           |          |             |          |
| CuPy (Fatbin)         |           |          |             |          |
| CuPy (Launch Bounds)  |           |          |             |          |
| CuPy (Trig Functions) |           |          |             |          |

In [ ]:
## GPU Scenario #4
1. CuPy implementation of Lombscargle function
2. Added user cache
3. Specific functions per data type
4. Utilize fast_math compiler flag

In [17]:
!nsys profile -s none -o cupy_v4 -f true python3 icassp_cupy_v4.py float32 2000 0

Registers 32
Processing events...
Capturing symbol files...
Saving temporary "/tmp/nsys-report-7b2a-1373-6f01-fa8e.qdstrm" file to disk...
Creating final output files...

Processing [==============================================================100%]
Saved report file to "/tmp/nsys-report-7b2a-1373-6f01-fa8e.qdrep"
Report file moved to "/home/odysseus/workStuff/cusignal-icassp-tutorial/notebooks/raw_cupy_cuda/cupy_v4.qdrep"


In [18]:
!nsys stats --report gpukernsum --report nvtxppsum cupy_v4.qdrep

Generate SQLite file cupy_v4.sqlite from cupy_v4.qdrep
Exporting 618014 events: [================================================100%]
Using cupy_v4.sqlite file for stats and reports.
Exporting [/opt/nvidia/nsight-systems/2020.3.2/target-linux-x64/reports/gpukernsum cupy_v4.sqlite] to console... 

 Time(%)  Total Time (ns)  Instances    Average     Minimum    Maximum             Name           
 -------  ---------------  ---------  -----------  ---------  ---------  -------------------------
   100.0    4,491,357,965      2,001  2,244,556.7  2,146,486  2,504,420  _cupy_lombscargle_float32

Exporting [/opt/nvidia/nsight-systems/2020.3.2/target-linux-x64/reports/nvtxppsum cupy_v4.sqlite] to console... 

 Time(%)  Total Time (ns)  Instances     Average       Minimum      Maximum            Range        
 -------  ---------------  ---------  -------------  -----------  -----------  ---------------------
    97.4    4,669,349,251      2,000    2,334,674.6    2,234,348    2,716,439  cupy_lom

### Single Precision
|                       | Registers | JIT (ms) | Kernel (ms) | Speed Up |
|:----------------------|:----------|:---------|:------------|:---------|
| Scipy                 | -         | 51,075.2 | 51,128.2    | 1.0      |
| CuPy (Baseline)       | 30        | 135.1    | 10.1        | 5062.2   |
| CuPy (User Cache)     | 30        | 137.2    | 10.1        | 5062.2   |
| CuPy (Data Type)      | 26        | 135.4    | 10.3        | 4963.9   |
| CuPy (Fast Math)      | 32        | 126.1    | 2.3         | 22229.7  |
| CuPy (Fatbin)         |           |          |             |          |
| CuPy (Launch Bounds)  |           |          |             |          |
| CuPy (Trig Functions) |           |          |             |          |

### Double Precision
|                       | Registers | JIT (ms) | Kernel (ms) | Speed Up |
|:----------------------|:----------|:---------|:------------|:---------|
| Scipy                 | -         | 50.799.1 | 50,806.4    | 1.0      |
| CuPy (Baseline)       | 48        | 145.0    | 21.2        | 2396.5   |
| CuPy (User Cache)     | 48        | 145.7    | 21.2        | 2396.5   |
| CuPy (Data Type)      | 48        | 150.4    | 21.2        | 2396.5   |
| CuPy (Fast Math)      | 48        | 146.1    | 21.2        | 2396.5   |
| CuPy (Fatbin)         |           |          |             |          |
| CuPy (Launch Bounds)  |           |          |             |          |
| CuPy (Trig Functions) |           |          |             |          |

## GPU Scenario #5
1. CuPy implementation of Lombscargle function
2. Added user cache
3. Specific functions per data type
4. Utilize fast_math compiler flag
5. Precompile CUDA kernels into fatbins to be loaded at runtime

In [41]:
!nvcc --fatbin -std=c++11 --use_fast_math \
    --generate-code arch=compute_70,code=sm_70 \
    --generate-code arch=compute_75,code=sm_75 \
    --generate-code arch=compute_80,code=[sm_80,compute_80] \
    _lombscargle.cu -odir .

In [22]:
!nsys profile -s none -o cupy_v5 -f true python3 icassp_cupy_v5.py float32 2000 0

Registers 32
Processing events...
Capturing symbol files...
Saving temporary "/tmp/nsys-report-30cb-87cc-852e-15cb.qdstrm" file to disk...
Creating final output files...

Processing [==============================================================100%]
Saved report file to "/tmp/nsys-report-30cb-87cc-852e-15cb.qdrep"
Report file moved to "/home/odysseus/workStuff/cusignal-icassp-tutorial/notebooks/raw_cupy_cuda/cupy_v5.qdrep"


In [23]:
!nsys stats --report gpukernsum --report nvtxppsum cupy_v5.qdrep

Generate SQLite file cupy_v5.sqlite from cupy_v5.qdrep
Exporting 625201 events: [================================================100%]
Using cupy_v5.sqlite file for stats and reports.
Exporting [/opt/nvidia/nsight-systems/2020.3.2/target-linux-x64/reports/gpukernsum cupy_v5.sqlite] to console... 

 Time(%)  Total Time (ns)  Instances    Average     Minimum    Maximum             Name           
 -------  ---------------  ---------  -----------  ---------  ---------  -------------------------
   100.0    4,494,166,736      2,001  2,245,960.4  2,149,112  2,510,050  _cupy_lombscargle_float32

Exporting [/opt/nvidia/nsight-systems/2020.3.2/target-linux-x64/reports/nvtxppsum cupy_v5.sqlite] to console... 

 Time(%)  Total Time (ns)  Instances    Average     Minimum    Maximum           Range        
 -------  ---------------  ---------  -----------  ---------  ---------  ---------------------
    99.9    4,689,032,098      2,000  2,344,516.0  2,250,261  2,625,194  cupy_lombscargle_loop
    

### Single Precision
|                       | Registers | JIT (ms) | Kernel (ms) | Speed Up |
|:----------------------|:----------|:---------|:------------|:---------|
| Scipy                 | -         | 51,075.2 | 51,128.2    | 1.0      |
| CuPy (Baseline)       | 30        | 135.1    | 10.1        | 5062.2   |
| CuPy (User Cache)     | 30        | 137.2    | 10.1        | 5062.2   |
| CuPy (Data Type)      | 26        | 135.4    | 10.3        | 4963.9   |
| CuPy (Fast Math)      | 32        | 126.1    | 2.3         | 22229.7  |
| CuPy (Fatbin)         | 32        | 3.9      | 2.3         | 22229.7  |
| CuPy (Launch Bounds)  |           |          |             |          |
| CuPy (Trig Functions) |           |          |             |          |

### Double Precision
|                       | Registers | JIT (ms) | Kernel (ms) | Speed Up |
|:----------------------|:----------|:---------|:------------|:---------|
| Scipy                 | -         | 50.799.1 | 50,806.4    | 1.0      |
| CuPy (Baseline)       | 48        | 145.0    | 21.2        | 2396.5   |
| CuPy (User Cache)     | 48        | 145.7    | 21.2        | 2396.5   |
| CuPy (Data Type)      | 48        | 150.4    | 21.2        | 2396.5   |
| CuPy (Fast Math)      | 48        | 146.1    | 21.2        | 2396.5   |
| CuPy (Fatbin)         | 48        | 22.3     | 21.2        | 2396.5   |
| CuPy (Launch Bounds)  |           |          |             |          |
| CuPy (Trig Functions) |           |          |             |          |

## GPU Scenario #6
1. CuPy implementation of Lombscargle function
2. Added user cache
3. Specific functions per data type
4. Utilize fast_math compiler flag
5. Precompile CUDA kernels into fatbins to be loaded at runtime
6. Add launch bounds to CUDA kernels

In [44]:
!nvcc --fatbin -std=c++11 --use_fast_math \
    --generate-code arch=compute_70,code=sm_70 \
    --generate-code arch=compute_75,code=sm_75 \
    --generate-code arch=compute_80,code=[sm_80,compute_80] \
    _lombscargle_lb.cu -odir .

In [27]:
!nsys profile -s none -o cupy_v6 -f true python3 icassp_cupy_v6.py float32 2000 0

Registers 31
Processing events...
Capturing symbol files...
Saving temporary "/tmp/nsys-report-d398-52e1-6e3c-39a7.qdstrm" file to disk...
Creating final output files...

Processing [==============================================================100%]
Saved report file to "/tmp/nsys-report-d398-52e1-6e3c-39a7.qdrep"
Report file moved to "/home/odysseus/workStuff/cusignal-icassp-tutorial/notebooks/raw_cupy_cuda/cupy_v6.qdrep"


In [28]:
!nsys stats --report gpukernsum --report nvtxppsum cupy_v6.qdrep

Generate SQLite file cupy_v6.sqlite from cupy_v6.qdrep
Exporting 644094 events: [================================================100%]
Using cupy_v6.sqlite file for stats and reports.
Exporting [/opt/nvidia/nsight-systems/2020.3.2/target-linux-x64/reports/gpukernsum cupy_v6.sqlite] to console... 

 Time(%)  Total Time (ns)  Instances    Average     Minimum    Maximum             Name           
 -------  ---------------  ---------  -----------  ---------  ---------  -------------------------
   100.0    4,502,552,069      2,001  2,250,151.0  2,153,668  2,498,410  _cupy_lombscargle_float32

Exporting [/opt/nvidia/nsight-systems/2020.3.2/target-linux-x64/reports/nvtxppsum cupy_v6.sqlite] to console... 

 Time(%)  Total Time (ns)  Instances    Average     Minimum    Maximum           Range        
 -------  ---------------  ---------  -----------  ---------  ---------  ---------------------
    99.9    4,686,917,005      2,000  2,343,458.5  2,247,224  2,707,435  cupy_lombscargle_loop
    

### Single Precision
|                       | Registers | JIT (ms) | Kernel (ms) | Speed Up |
|:----------------------|:----------|:---------|:------------|:---------|
| Scipy                 | -         | 51,075.2 | 51,128.2    | 1.0      |
| CuPy (Baseline)       | 30        | 135.1    | 10.1        | 5062.2   |
| CuPy (User Cache)     | 30        | 137.2    | 10.1        | 5062.2   |
| CuPy (Data Type)      | 26        | 135.4    | 10.3        | 4963.9   |
| CuPy (Fast Math)      | 32        | 126.1    | 2.3         | 22229.7  |
| CuPy (Fatbin)         | 32        | 3.9      | 2.3         | 22229.7  |
| CuPy (Launch Bounds)  | 31        | 3.9      | 2.3         | 22229.7  |
| CuPy (Trig Functions) |           |          |             |          |

### Double Precision
|                       | Registers | JIT (ms) | Kernel (ms) | Speed Up |
|:----------------------|:----------|:---------|:------------|:---------|
| Scipy                 | -         | 50.799.1 | 50,806.4    | 1.0      |
| CuPy (Baseline)       | 48        | 145.0    | 21.2        | 2396.5   |
| CuPy (User Cache)     | 48        | 145.7    | 21.2        | 2396.5   |
| CuPy (Data Type)      | 48        | 150.4    | 21.2        | 2396.5   |
| CuPy (Fast Math)      | 48        | 146.1    | 21.2        | 2396.5   |
| CuPy (Fatbin)         | 48        | 22.3     | 21.2        | 2396.5   |
| CuPy (Launch Bounds)  | 48        | 22.8     | 21.2        | 2396.5   |
| CuPy (Trig Functions) |           |          |             |          |

## GPU Scenario #7
1. CuPy implementation of Lombscargle function
2. Added user cache
3. Specific functions per data type
4. Utilize fast_math compiler flag
5. Precompile CUDA kernels into fatbins to be loaded at runtime
6. Add launch bounds to CUDA kernels
7. Use fast sin + cos functions

In [50]:
!nvcc --fatbin -std=c++11 --use_fast_math \
    --generate-code arch=compute_70,code=sm_70 \
    --generate-code arch=compute_75,code=sm_75 \
    --generate-code arch=compute_80,code=[sm_80,compute_80] \
    _lombscargle_lb_trig.cu -odir .

In [37]:
!nsys profile -s none -o cupy_v7 -f true python3 icassp_cupy_v7.py float32 2000 0

Registers 31
Processing events...
Capturing symbol files...
Saving temporary "/tmp/nsys-report-08cb-6736-c348-211c.qdstrm" file to disk...
Creating final output files...

Processing [==============================================================100%]
Saved report file to "/tmp/nsys-report-08cb-6736-c348-211c.qdrep"
Report file moved to "/home/odysseus/workStuff/cusignal-icassp-tutorial/notebooks/raw_cupy_cuda/cupy_v7.qdrep"


In [38]:
!nsys stats --report gpukernsum --report nvtxppsum cupy_v7.qdrep

Generate SQLite file cupy_v7.sqlite from cupy_v7.qdrep
Exporting 646937 events: [================================================100%]
Using cupy_v7.sqlite file for stats and reports.
Exporting [/opt/nvidia/nsight-systems/2020.3.2/target-linux-x64/reports/gpukernsum cupy_v7.sqlite] to console... 

 Time(%)  Total Time (ns)  Instances    Average     Minimum    Maximum             Name           
 -------  ---------------  ---------  -----------  ---------  ---------  -------------------------
   100.0    4,470,477,831      2,001  2,234,121.9  2,132,962  2,577,603  _cupy_lombscargle_float32

Exporting [/opt/nvidia/nsight-systems/2020.3.2/target-linux-x64/reports/nvtxppsum cupy_v7.sqlite] to console... 

 Time(%)  Total Time (ns)  Instances    Average     Minimum    Maximum           Range        
 -------  ---------------  ---------  -----------  ---------  ---------  ---------------------
    99.9    4,651,871,063      2,000  2,325,935.5  2,223,617  2,668,839  cupy_lombscargle_loop
    

### Single Precision
|                       | Registers | JIT (ms) | Kernel (ms) | Speed Up |
|:----------------------|:----------|:---------|:------------|:---------|
| Scipy                 | -         | 51,075.2 | 51,128.2    | 1.0      |
| CuPy (Baseline)       | 30        | 135.1    | 10.1        | 5062.2   |
| CuPy (User Cache)     | 30        | 137.2    | 10.1        | 5062.2   |
| CuPy (Data Type)      | 26        | 135.4    | 10.3        | 4963.9   |
| CuPy (Fast Math)      | 32        | 126.1    | 2.3         | 22229.7  |
| CuPy (Fatbin)         | 32        | 3.9      | 2.3         | 22229.7  |
| CuPy (Launch Bounds)  | 31        | 3.9      | 2.3         | 22229.7  |
| CuPy (Trig Functions) | 31        | 3.9      | 2.3         | 22229.7  |

### Double Precision
|                       | Registers | JIT (ms) | Kernel (ms) | Speed Up |
|:----------------------|:----------|:---------|:------------|:---------|
| Scipy                 | -         | 50.799.1 | 50,806.4    | 1.0      |
| CuPy (Baseline)       | 48        | 145.0    | 21.2        | 2396.5   |
| CuPy (User Cache)     | 48        | 145.7    | 21.2        | 2396.5   |
| CuPy (Data Type)      | 48        | 150.4    | 21.2        | 2396.5   |
| CuPy (Fast Math)      | 48        | 146.1    | 21.2        | 2396.5   |
| CuPy (Fatbin)         | 48        | 22.3     | 21.2        | 2396.5   |
| CuPy (Launch Bounds)  | 48        | 22.8     | 21.2        | 2396.5   |
| CuPy (Trig Functions) | 48        | 15.5     | 14.1        | 3603.3   |